In [1]:
!curl 127.0.0.1:5000 

{
    "task": "Analyze images. Input is an image file and output is json-like with raw data. Usage: curl -i -F \"image=@<full_image_path>\" yenthenas.ddns.net:5000"
}


In [1]:
!curl -i -F "image=@/home/yuuhanase/workspace/hackhcmc/hackhcmc_dataset/z4550859993864_d78b088221eb80075d1256e0f18991ad.jpg" 127.0.0.1:5000 

HTTP/1.1 200 OK
Server: Werkzeug/3.0.3 Python/3.10.12
Date: Sat, 29 Jun 2024 10:04:38 GMT
Content-Type: application/json
Content-Length: 4612
Connection: close

{
    "product": {
        "0": {
            "brand": "bia viet",
            "label": "carton box",
            "loc": {
                "x1": 0,
                "y1": 152,
                "x2": 151,
                "y2": 274
            }
        },
        "1": {
            "brand": "heineken",
            "label": "carton box",
            "loc": {
                "x1": 612,
                "y1": 534,
                "x2": 727,
                "y2": 628
            }
        },
        "2": {
            "label": "beer bottle",
            "loc": {
                "x1": 323,
                "y1": 352,
                "x2": 397,
                "y2": 596
            },
            "brand": "heineken"
        },
        "3": {
            "label": "beer",
            "loc": {
                "x1": 324,
                "y1":

In [4]:
import requests

requests.get('http://192.168.3.122:5000/')

<Response [200]>

In [7]:
import requests
import os

url = 'http://127.0.0.1:5000/'
path_img = '/home/yuuhanase/workspace/hackhcmc/hackhcmc_dataset/z4550859993864_d78b088221eb80075d1256e0f18991ad.jpg'

with open(path_img, 'rb') as img:
    name_img= os.path.basename(path_img)
    files= {'image': (name_img,img,'multipart/form-data',{'Expires': '0'}) }
    with requests.Session() as s:
        r = s.post(url,files=files)
        print(r.status_code)
  

200


In [8]:
from pprint import pprint
pprint(r.json())

{'context': {'action': 'celebration',
             'context': 'pub',
             'environment': 'outdoor'},
 'human': {'0': {'action': 'talking',
                 'emotion': 'happy',
                 'gender': 'male',
                 'loc': {'x1': 786, 'x2': 949, 'y1': 518, 'y2': 767},
                 'role': 'drinker'},
           '1': {'action': 'smiling',
                 'emotion': 'happy',
                 'gender': 'male',
                 'loc': {'x1': 446, 'x2': 502, 'y1': 404, 'y2': 568},
                 'role': 'buyer'},
           '2': {'action': 'drinking',
                 'emotion': 'happy',
                 'gender': 'male',
                 'loc': {'x1': 901, 'x2': 1023, 'y1': 508, 'y2': 759},
                 'role': 'seller'},
           '3': {'action': 'eating',
                 'emotion': 'angry',
                 'gender': 'male',
                 'loc': {'x1': 999, 'x2': 1018, 'y1': 459, 'y2': 503},
                 'role': 'seller'}},
 'posm': {'0': {'brand':

In [9]:
import json

with open('data.json', 'w') as f:
    json.dump(r.json(), f)